In [4]:
import nltk
from tools import show_subtitle

# Ch8 分析句子结构

前面的章节重点关注词的处理：识别、分析结构、分配词汇类别、获取词汇的含义、识别词序列或者n-grams的模式

本章的学习目标：

1.  使用形式化语法来描述无限的句子集合的结构
2.  使用句法树来表示句子结构
3.  使用解析器来分析句子，并且自动构建语法树


## 8.1 一些语法困境

### 8.1.1 语言数据和无限的可能性

本章中将采用“生成文法”的形式化框架，其中一种“语言”被认为仅仅是所有合乎文法的句子的大集合，而文法只是一个形式化符号，可以用于“生成”这个集合的成员。

文法的目的是给出一个明确的语言描述，使用 `S → S and S` 形式的递归产生式

### 8.1.2 普遍存在的歧义

In [5]:
groucho_grammar = nltk.CFG.fromstring('''
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | "I"
VP -> V NP | VP PP
Det -> "an" | "my"
N -> "elephant" | "pajamas"
V -> "shot"
P -> "in"
''')

In [6]:
# 基于一种文法解析句子，可能会解析出两种结构
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)  # 图解析
for i, tree in enumerate(parser.parse(sent)):
    show_subtitle(f"第 {i+1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
--------------- >第 2 个结构< ---------------
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## 8.2 文法的用途

### 8.2.1 超越 n-grams

-   成分结构：是词与词结合在一起组成的单元。
-   词汇的可替代性：在符合语法规则的句子中，长的词序列可以被一个更短的词序列替代，并且替代后不会导致句子不合语法规则
    -   形成单元的每个序列都可以被单独的词替换。
    -   短语的方法类别标签
        -   名词短语（NP）
        -   动词短语（VP）
        -   介词短语（PP）

句子长度是任意的，因此短语结构树的深度也是任意的。因为Sec7.4只能产生有限深度的结构，所以分块方法并不适合用于句法分析。


## 8.3 上下文无关文法（context-free grammars，CFG）

### 8.3.1 一种简单的文法

In [7]:
# Ex8-1 一个简单的上下文无关文法的例子
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my" | "The"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

In [8]:
# 句子剖析会出现两个符合文法规则的结果，称为结构上有歧义。这个歧义称为介词短语附着歧义。
sent = 'The dog saw a man in the park'.split()
rd_parser = nltk.RecursiveDescentParser(grammar1)  # 递归下降解析器 RecursiveDescentParser()
for i, tree in enumerate(rd_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man) (PP (P in) (NP (Det the) (N park))))))
--------------- >第 2 个结构< ---------------
(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man))
    (PP (P in) (NP (Det the) (N park)))))


In [9]:
nltk.app.rdparser()  # 通过这个演示可以辅助理解从顶向下的回溯策略的句法剖析过程

### 8.3.2 编写自己的文法

In [11]:
# 在文本文件创建和编辑语法会更加文法，然后可以利用函数加载到NLTK中进行解析
grammar1 = nltk.data.load('mygrammar.cfg')
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)  # trace = 2 不知道有何作用
for i, tree in enumerate(rd_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S (NP Mary) (VP (V saw) (NP Bob)))


In [13]:
# 输出文法文件中的内容
for p in grammar1.productions():
    print(p)

S -> NP VP
VP -> V NP
VP -> V NP PP
PP -> P NP
V -> 'saw'
V -> 'ate'
V -> 'walked'
NP -> 'John'
NP -> 'Mary'
NP -> 'Bob'
NP -> Det N
NP -> Det N PP
Det -> 'a'
Det -> 'an'
Det -> 'the'
Det -> 'my'
Det -> 'The'
N -> 'man'
N -> 'dog'
N -> 'cat'
N -> 'telescope'
N -> 'park'
P -> 'in'
P -> 'on'
P -> 'by'
P -> 'with'


### 8.3.3 句法结构中的递归

RecursiveDescentParser()无法处理形如X→XY的左递归产生式

In [14]:
# Ex-2：递归的上下文无关文法
grammar2 = nltk.CFG.fromstring("""
S  -> NP VP
NP -> Det Nom | PropN
Nom -> Adj Nom | N
VP -> V Adj | V NP | V S | V NP PP
PP -> P NP
PropN -> 'Buster' | 'Chatterer' | 'Joe'
Det -> 'the' | 'a'
N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
P -> 'on'
""")

In [16]:
sent = 'the angry bear chased the frightened little squirrel'.split()
# RecursiveDescentParser()无法处理形如X→XY的左递归产生式
rd_parser = nltk.RecursiveDescentParser(grammar2)
for i, tree in enumerate(rd_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S
  (NP (Det the) (Nom (Adj angry) (Nom (N bear))))
  (VP
    (V chased)
    (NP
      (Det the)
      (Nom (Adj frightened) (Nom (Adj little) (Nom (N squirrel)))))))


In [15]:
sent = 'Chatterer said Buster thought the tree was tall'.split()
rd_parser = nltk.RecursiveDescentParser(grammar2)
for i, tree in enumerate(rd_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S
  (NP (PropN Chatterer))
  (VP
    (V said)
    (S
      (NP (PropN Buster))
      (VP
        (V thought)
        (S (NP (Det the) (Nom (N tree))) (VP (V was) (Adj tall)))))))


## 8.4 上下文无关文法分析

解析器根据文法产生式处理输入的句子，并且建立一个或者多个符合文法的组成结构

文法是一个格式良好的声明规则——实际上只是一个字符串，而不是程序。

解析器是文法的解释程序，用于搜索所有的符合文法的树的空间，并且找出一棵与句子匹配的语法树

两种分析算法：

1.  自顶向下的递归下降分析，主要缺点：
    -   左递归产生式，如：NP→NP PP，会进入死循环
    -   解析器在处理不符合输入句子的词和结构时会浪费许多时间
    -   回溯过程中可能会丢弃分析过的成分，需要再次重建
2.  自底向上的移进归约分析，只建立与输入中的词对应的结构，对于每个子结构只建立一次
    -   反复将下一个输入词捡到堆栈，叫做移位操作
    -   替换前n项为一项的操作，叫做归约操作

「移进——归约」解析器 ShiftReduceParser()

In [19]:
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my" | "The"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

In [21]:
sent = 'Mary saw a dog'.split()
sr_parser = nltk.ShiftReduceParser(grammar1)
for i, tree in enumerate(sr_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

--------------- >第 1 个结构< ---------------
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [22]:
# 跟踪解析的过程
sent = 'Mary saw a dog'.split()
sr_parser = nltk.ShiftReduceParser(grammar1,trace = 2)
for i, tree in enumerate(sr_parser.parse(sent)):
    show_subtitle(f"第 {i + 1} 个结构")
    print(tree)

Parsing 'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
--------------- >第 1 个结构< ---------------
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [23]:
nltk.app.srparser()  # 通过这个演示可以辅助理解自底向上的称进归约的句法剖析过程

### 8.4.3 左角落解析器

是 自顶向下 和 自底向上 方法的混合体，是一个带有自底向上过滤的自顶向下的解析器

左角落解析器，不会陷入左递归产生式死循环。

### 8.4.4 符合语句规则的子串表（WFST）

上述简单的解析器都存在完整性和效率问题，下面将基于图表分析：即利用动态规划算法来解决这些问题

动态规划算法存储中间结果，并且在适当的时候重用，从而显著提高了效率。

WFST的缺点：
-   WFST本身不是一个分析树
-   每个非词汇文法生产式都必须是二元的
-   作为一个自下而上的文法，潜在地存在着浪费，因为它会在不符合文法的地方提出成分，后面又会放弃掉错误的成分
-   WFST并不能表示句子中的结构歧义（如两个动词短语的读取）


In [30]:
# 可以在文法中直接查找文本中单词所属类别
# lhs : left-hand-side; rhs : right-hand-side
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
productions = groucho_grammar.productions(rhs=text[3])
for product in productions:
    print(product)
    print(product.lhs())
    print(product.rhs())

N -> 'elephant'
N
('elephant',)
